In [ ]:
#G050.2213-00.6063
#19:25:57.52 15:03:00.3
#3.3 kpc
#Band 6 ALMA
#CH3OH (methanol)

In [ ]:
# For Colab
!pip install peakutils
!pip install lineid_plot
!pip install astroquery
!pip install lmfit
!wget https://github.com/saint-germain/rot_diag/raw/main/G050.2213-00.6063.csv

In [ ]:
from astroquery.linelists.cdms import CDMS
import astropy.units as u
import astropy.constants as c
import peakutils
from peakutils.plot import plot as pplot
import matplotlib.pyplot as plt
import pandas as pd
import lineid_plot
from lmfit.models import GaussianModel
import numpy as np
from scipy.optimize import curve_fit

In [ ]:
# search in CDMS database for plausible CH3OH lines in relevant range (see slides)
min_frequency=241.67
max_frequency=241.91
response = CDMS.query_lines(min_frequency= min_frequency* u.GHz,
                            max_frequency=max_frequency * u.GHz,
                            molecule="032504 CH3OH, vt=0-2",
                            get_query_payload=False,temperature_for_intensity=0)

In [ ]:
line_wave=response['FREQ']/1e3 # MHz to GHz
line_label=[str(i) for i in line_wave] # line label for plotting

In [ ]:
fname='G050.2213-00.6063.csv'
df = pd.read_csv(fname)
freq=df['Freq'] # in GHz
i_nu=df['Intensity'] # in Jy sr^-1

In [ ]:
# plot cropped spectrum + plausible line locations
# peakutils is only used here as a visual aid

filter=(freq>min_frequency)&(freq<max_frequency)
freqn=freq[filter]
i_nun=i_nu[filter]

indices = peakutils.indexes(i_nun, thres=0.25, min_dist=0.1)  # indices are the "peak" positions
fig, ax = plt.subplots(figsize=(20,10),dpi=150)

pplot(freqn, i_nun, indices)
ax.set_xlim(min_frequency,max_frequency)
ax.set_ylim(-4e11,0.8e12)
lineid_plot.plot_line_ids(freqn, i_nun, line_wave, line_label, ax=ax, arrow_tip=0, box_loc=-3e11, max_iter=300, label1_size=6) # locate theoretical lines from cdms query
ax.set_xlabel('Frequency (GHz)')
ax.set_ylabel('Intensity (Jy sr^-1)')

In [ ]:
# make a gaussian model for each plausible line using lmfit
for i,j in zip(line_wave,range(len(line_wave))):
  gausstemp=GaussianModel(prefix='g'+str(j)+'_')

npeaks=len(line_wave)
model=GaussianModel(prefix='g1_')
for i in range(1,npeaks):
  model=model+GaussianModel(prefix='g%d_' % (i+1))
pars=model.make_params()
for i,ff in zip(range(npeaks),line_wave):
  pars['g%d_center' % (i+1)].set(value=ff,vary=False) # fix nu_ul
  pars['g%d_sigma' % (i+1)].set(value=0.01, min=1e-3,max=0.02)
  pars['g%d_amplitude' % (i+1)].set(value=0.01, min=0,max=5e14)

In [ ]:
out=model.fit(i_nun,pars,x=freqn) # run fitting algorithm
comps = out.eval_components(x=freqn) # fit results for each line

In [ ]:
# plot left: data,model,residuals
# plot right: plot each line fit
x=freqn
y=i_nun

fig, axes = plt.subplots(1, 2, figsize=(12.8, 4.8))
axes[0].plot(x, y, 'b',label='Spectrum')
axes[0].plot(x, out.best_fit, 'r-', label='Best fit')
axes[0].plot(x, out.residual, 'k--', label='Residual')
axes[0].legend(loc='best')
axes[0].set_xlabel('Frequency (GHz)')
axes[0].set_ylabel('Intensity (Jy sr^-1)')


axes[1].plot(x, y, 'b',label='Spectrum')
for i in range(npeaks):
  axes[1].plot(x, comps['g%d_' % (i+1)], label='g'+str(i+1))

axes[1].legend(loc='center left', bbox_to_anchor=(1,0.5))
axes[1].set_xlabel('Frequency (GHz)')
axes[1].set_ylabel('Intensity (Jy sr^-1)')

plt.tight_layout()

In [ ]:
# save line parameters:
# frequency, eup, aul, gup, i dnu integral
# GHz, K, s^-1, N/A, Jy*sr^-1*GHz
elo=response['ELO']*1.43*u.K*u.cm # cm**-1 to K
etrans=line_wave*0.04799*u.K/u.MHz # GHz**-1 to K
eup=elo+etrans

Aul=10**response['LGAIJ']
gup=response['GUP']

dnu=freqn.iloc[1]-freqn.iloc[0]
integ_txt=[np.trapz(comps['g%d_'%(i+1)],dx=dnu) for i in range(npeaks)]

mydict={'Freq':line_wave,'Eup':eup,'Aul':Aul,'gup':gup,'I dv':integ_txt}
df_txt = pd.DataFrame(mydict)
df_txt.to_csv('line_parameters.csv', index=False)

In [ ]:
# GOAL A
# use parameters to calculate N_u in cm^-2
# make rotational diagram (log-log)
# eup < 500 K; log(n_u/g_ul) > 26
# fit straight line
# use slope to calculate T in K, T ~ 15 K

In [ ]:
# get partition function to obtain N_CH3OH
result = CDMS.get_species_table()
mol = result[result['tag'] == 32504]
mol.pprint(max_width=160)

In [ ]:
# fit interpolation function to partition function data
# check that it's ok near T

In [ ]:
# GOAL B
# use interpolated partition function to calculate N_CH3OH ~ 1.5e15 cm^-2

